In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_cost = pd.read_csv('../data/raw_data/costs_us.csv')
df_orders = pd.read_csv('../data/raw_data/orders_log_us.csv')
df_visits = pd.read_csv('../data/raw_data/visits_log_us.csv')


### Descrição dos Dados

O projeto utiliza três fontes de dados para analisar a performance de marketing:

*   **Tabela `visits` (Logs de Acesso):**
    *   `Uid`: ID do usuário
    *   `Device`: Dispositivo (desktop/touch)
    *   `Start Ts`: Início da sessão (data/hora)
    *   `End Ts`: Fim da sessão (data/hora)
    *   `Source Id`: ID da fonte de tráfego

*   **Tabela `orders` (Dados de Pedidos):**
    *   `Uid`: ID do usuário
    *   `Buy Ts`: Data e hora do pedido
    *   `Revenue`: Receita do pedido

*   **Tabela `costs` (Custos de Marketing):**
    *   `source_id`: ID da fonte de anúncio
    *   `dt`: Data do gasto
    *   `costs`: Valor gasto

In [3]:
df_cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   source_id  2542 non-null   int64  
 1   dt         2542 non-null   object 
 2   costs      2542 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 59.7+ KB


In [4]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Buy Ts   50415 non-null  object 
 1   Revenue  50415 non-null  float64
 2   Uid      50415 non-null  uint64 
dtypes: float64(1), object(1), uint64(1)
memory usage: 1.2+ MB


In [5]:
df_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Device     359400 non-null  object
 1   End Ts     359400 non-null  object
 2   Source Id  359400 non-null  int64 
 3   Start Ts   359400 non-null  object
 4   Uid        359400 non-null  uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 13.7+ MB


In [6]:
# Renomeando as colunas
df_cost.rename(columns={'source_id': 'id', 'dt': 'date', 'costs': 'cost'}, inplace=True)
df_orders.rename(columns={'Uid': 'uid', 'Buy Ts': 'buy_date', 'Revenue': 'revenue'}, inplace=True)
df_visits.rename(columns={'Uid': 'uid', 'Device': 'device', 'Start Ts': 'start_date', 'End Ts': 'end_date', 'Source Id': 'id'}, inplace=True)

In [7]:
# Adequação dos tipos 

# df_cost
# Convertendo a coluna 'date' para datetime
df_cost['date'] = pd.to_datetime(df_cost['date'])

# df_orders
# Convertendo a coluna 'buy_date' para datetime
df_orders['buy_date'] = pd.to_datetime(df_orders['buy_date'])

# df_visits
# Convertendo as colunas de data para datetime
df_visits['start_date'] = pd.to_datetime(df_visits['start_date'])
df_visits['end_date'] = pd.to_datetime(df_visits['end_date'])

# Convertendo a coluna 'device' para category
df_visits['device'] = df_visits['device'].astype('category')

In [8]:
# Verificação de Nulos 
print("Valores Nulos em df_cost:")
print(df_cost.isnull().sum())
print("\nValores Nulos em df_orders:")
print(df_orders.isnull().sum())
print("\nValores Nulos em df_visits:")
print(df_visits.isnull().sum())


Valores Nulos em df_cost:
id      0
date    0
cost    0
dtype: int64

Valores Nulos em df_orders:
buy_date    0
revenue     0
uid         0
dtype: int64

Valores Nulos em df_visits:
device        0
end_date      0
id            0
start_date    0
uid           0
dtype: int64


In [12]:
[df.to_csv(f'../data/processed_data/{name}.csv', index=False) for name, df in zip(['costs', 'orders', 'visits'], [df_cost, df_orders, df_visits])]

[None, None, None]


### Processamento e Limpeza dos Dados

Nesta fase inicial, os dados brutos foram preparados para a análise, garantindo sua qualidade e consistência. O processo foi dividido em três etapas principais:

**1. Carga dos Dados:**
*   Foram carregados três arquivos CSV: `visits_log_us.csv`, `orders_log_us.csv` e `costs_us.csv`, utilizando a biblioteca Pandas.

**2. Limpeza e Adequação:**
*   **Renomeação das Colunas:** Os nomes das colunas foram padronizados para facilitar o acesso e a manipulação (ex: `Uid` para `uid`, `Start Ts` para `start_date`).
*   **Ajuste de Tipos:** Todas as colunas contendo datas e horas foram convertidas para o formato `datetime`, permitindo análises temporais precisas.
*   **Otimização:** A coluna `device` foi convertida para o tipo `category` para otimizar o uso de memória.

**3. Validação dos Dados:**
*   **Verificação de Nulos e Duplicatas:** Foi realizada uma checagem final para confirmar a ausência de valores nulos ou linhas duplicadas em todos os conjuntos de dados. O resultado confirmou que os dados estão completos e íntegros.


In [ ]:
# Salvando dados
df_final.to_csv('.' \
                
                dados_finais_processados.csv', index=False)

NameError: name 'df_final' is not defined

Arquivos 'orders_cleaned.csv' e 'costs_cleaned.csv' criados com sucesso.
Tamanho da tabela de pedidos: (50415, 4)
Tamanho da tabela de custos: (12, 2)
